# Построение графика эмоциональной окраски книг

В данном ноутбуке реализован анализ настроения книги на основе словаря [Hedonometer](https://en.wikipedia.org/wiki/Hedonometer). Цель проекта — визуализировать динамику эмоционального фона текста, отслеживая «настроение» фрагментов произведения. Такой подход позволяет выявить ключевые эмоциональные моменты сюжета, увидеть пики и спады настроения, а также сделать качественные выводы о психологической динамике персонажей и событий.

Почему выбран Hedonometer, а не трансформеры

- Высокая скорость. Lexicon-based подход не требует вычислительных мощностей GPU и работает в разы быстрее по сравнению с трансформерами или другими нейросетевыми моделями.

- Сложность внедрения минимальна. Используется предрассчитанный csv файл.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from tqdm import tqdm
from razdel import sentenize
import re
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go

## Hedonometer модель

Загрузим и объединим русский и английский словарь Hedonometer

In [2]:
ru_words = pd.read_csv("Hedonometer_ru.csv", index_col=0)
en_words = pd.read_csv("Hedonometer_en.csv", index_col=0)
words_dict = dict(tuple(zip(ru_words['Word'], ru_words['Happiness Score'])) + tuple(zip(en_words['Word'], en_words['Happiness Score'])))
len(words_dict)

19455

Удаление стоп слов не используется, т.к. даже нейтральные слова меняют эмоциональный оттенок текста, уравновешивают окрашенные слова

In [3]:
def calculate_sentiment(text):
    cleaned = re.sub(r'[^A-Za-zА-Яа-яЁё]', ' ', text).lower()
    tokens = cleaned.split()
    words = [word for word in tokens if word in words_dict]
    if not words:
        return 5.
    word_freq = Counter(words)
    total_score = sum(words_dict[word] * freq for word, freq in word_freq.items())
    total_count = sum(word_freq.values())
    return total_score / total_count

## Загрузка и предобработка книги. Деление на чанки

In [4]:
with open("hobbit.txt", "r") as f:
    text = f.read().strip().replace("\xa0", " ").replace("…", "...")

Текст делится на чанки с равным количеством предложений

In [5]:
import re
from tqdm import tqdm

LENGTH=20

def split_text(text: str, n: int):
    sentences = [sent.text for sent in sentenize(text)]

    chunks = []
    for i in range(0, len(sentences), n):
        chunk = ' '.join(sentences[i:i + n])
        chunks.append(chunk)
    
    return chunks

chunks = split_text(text, LENGTH)
chunks[0][:200]


'Жил-был в норе под землей хоббит. Не в какой-то там мерзкой грязной сырой норе, где со всех сторон торчат хвосты червей и противно пахнет плесенью, но и не в сухой песчаной голой норе, где не на что с'

## Построение графика

In [6]:
scores = [calculate_sentiment(i) for i in chunks]

In [7]:
def moving_average(data, window_size):
    series = pd.Series(data)
    rolling_avg = series.rolling(window=window_size, center=True, min_periods=1).mean()
    return rolling_avg

In [8]:
WINDOW_SIZE = 5

df = pd.DataFrame({
    "index": list(range(len(chunks))),
    "score": scores,
    "smooth_score": moving_average(scores, WINDOW_SIZE),
    "text": chunks
})

In [9]:
fig = go.Figure()
# Сырое значение
fig.add_trace(go.Scatter(
    x=df['index'], y=df['score'],
    mode='markers', name='Raw Score',
    marker=dict(size=5, color='aquamarine')
))
# Усреднённая линия
fig.add_trace(go.Scatter(
    x=df['index'], y=df['smooth_score'],
    mode='lines', name=f'Rolling Mean (win={11})',
    line=dict(color='blue', dash='dash'),
))

fig.update_layout(
    title='Sentiment Analysis: Raw vs Rolling Average',
    xaxis_title='Fragment Index',
    yaxis_title='Sentiment Score',
    hovermode='x unified'
)

# Dash-приложение
app = dash.Dash(__name__)
app.title = "Sentiment Book Viewer"
app.layout = html.Div([
    html.H1("📖 Анализ настроения книги", style={'textAlign': 'center'}),
    dcc.Graph(id='sentiment-graph', figure=fig),
    html.H3("Выбранный фрагмент:"),
    html.Div(id='text-output', style={
        'whiteSpace': 'pre-wrap',
        'border': '1px solid #ccc',
        'padding': '1em',
        'backgroundColor': '#f9f9f9'
    })
])

@app.callback(
    Output('text-output', 'children'),
    Input('sentiment-graph', 'clickData')
)
def display_text(clickData):
    if clickData is None:
        return "Кликни на точку, чтобы увидеть фрагмент текста."
    idx = clickData['points'][0]['x']
    return chunks[int(idx)]


app.run(debug=False)
